In [1]:
2+5

7

In [2]:
from bs4 import BeautifulSoup
import lxml
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import re 
import pandas as pd

In [3]:
# cService = webdriver.ChromeService(executable_path='F:\Work\Automation Web scraping\Google maps automation\chromedriver-win64/chromedriver.exe')
# driver = webdriver.Chrome(service = cService)
# driver.get('https://www.google.com/search?sca_esv=1eda6e0add178775&tbs=lf:1,lf_ui:9&tbm=lcl&sxsrf=ADLYWIKKfoftf2ZwPJB7hwPYhLfu-QM08g:1721192815527&q=restaurant+behala&rflfq=1&num=10&sa=X&sqi=2&ved=2ahUKEwjR7O_ypq2HAxV1zDgGHTJcALQQjGp6BAgmEAE&biw=1536&bih=747&dpr=1.25#rlfi=hd:;si:;mv:[[22.510946,88.3279328],[22.4694611,88.30611859999999]]')
# time.sleep(2)

driver = webdriver.Chrome()
driver.get('https://www.google.com/search?q=travel+agents+in+mumbai&sca_esv=1eda6e0add178775&biw=1536&bih=695&tbm=lcl&ei=xsr8ZuWIM6bo1e8P2PaKwAI&oq=travel+agent+in+mumbai&gs_lp=Eg1nd3Mtd2l6LWxvY2FsIhZ0cmF2ZWwgYWdlbnQgaW4gbXVtYmFpKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIKEAAYFhgKGB4YDzIGEAAYFhgeMgYQABgWGB4yBhAAGBYYHjIGEAAYFhgeSKI1UABY1CxwAngAkAEAmAGnAqABiyqqAQYwLjkuMTW4AQPIAQD4AQGYAhqgAvgrwgIKEAAYgAQYQxiKBcICCxAAGIAEGLEDGIMBwgIIEAAYgAQYsQPCAgUQABiABMICCxAAGIAEGJECGIoFwgIQEAAYgAQYsQMYQxiDARiKBcICDRAAGIAEGLEDGEMYigXCAggQABiABBiSA8ICCxAAGIAEGJIDGIoFwgIIEAAYgAQYyQOYAwCSBwgyLjcuMTYuMaAHl6gB&sclient=gws-wiz-local#rlfi=hd:;si:;mv:[[19.216156899999998,72.9230578],[18.919284100000002,72.784847]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2')
time.sleep(2)

In [16]:
def find_emails(html_string):
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    emails = re.findall(email_pattern, html_string)
    return emails

soup = BeautifulSoup(driver.page_source, 'lxml')
locations = soup.find_all('div', jscontroller = 'AtSb') 
clicks = driver.find_elements(By.CLASS_NAME,'cXedhc')

companyArray = []
phoneArray = []
websiteArray = []
emailArray = []

count = 0 
while count < len(locations) :

    location = ''

    try:
        location = locations[count].text
        location = location[:location.index('.')-1]
    except:
        location = ''
    
    if "Sponsored" in location:
        print("Skipped")
    else:
        clicks[count].click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        phoneNumber = "No number present"
        try:
              phoneNumber = soup.find('a', jscontroller = 'LWZElb').text
        except:
              phoneNumber = "No number present"
        try:    
            website = soup.find('a', class_ = 'n1obkb mI8Pwc').get("href")
        except:
            website = ''
        
        if not website:
            companyArray.append(location)
            phoneArray.append(phoneNumber)
            websiteArray.append("No website present")
            emailArray.append("No Email")
            print(location +", "+ phoneNumber +", No website present"+", No Email")
        else:
            try:
                html_content = requests.get(website,verify=False).text
                soup = BeautifulSoup(html_content,"html.parser") 
                stringSoup = str(soup)
                emails = find_emails(stringSoup)
                print(location +", "+ phoneNumber +", "+ website +", ")
                if len(emails) > 0:
                    print(list(set(emails)))
                else:
                    print("No email found, search manually")
            except:
                  print("An exception occurred")
            companyArray.append(location)
            phoneArray.append(phoneNumber)
            websiteArray.append(website)
            try:
                emailArray.append(list(set(emails)))
            except:
                print("An exception occurred")
            print("----------------------------------------------------------------------------------------")
            
    count += 1

data = {
    'Company': companyArray,
    'Phone': phoneArray,
    'Website': websiteArray,
    'Email': emailArray,
}

df = pd.DataFrame(data)
df.to_excel('output_data.xlsx', index=False)
print("Data exported to output_data.xlsx successfully!")

RadhaKrishna Wedding Decorators & Caterers, 092214 01413, No website present, No Email


C:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'weandyouevents.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.weandyouevents.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


, 093211 43633, http://weandyouevents.com/, 
['8c4075d5481d476e945486754f783364@sentry.io', '8eb368c655b84e029ed79ad7a5c1718e@sentry.wixpress.com', 'eventsbyweandyou@gmail.com', '1eeb89147c984dc6bc3ffafd9e6cd089@sentry.wixpress.com', 'askweandyouevents@gmail.com', '605a7baede844d278b89dc95ae0a9123@sentry-next.wixpress.com']
----------------------------------------------------------------------------------------


C:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thoughtfactoryweb.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Thought Factory, 074000 66894, https://thoughtfactoryweb.com/, 
No email found, search manually
----------------------------------------------------------------------------------------
DIAN, 077383 77889, http://dianindia.com/, 
No email found, search manually
----------------------------------------------------------------------------------------
An exception occurred
----------------------------------------------------------------------------------------
Data exported to output_data.xlsx successfully!


In [4]:
# #code backup
# soup = BeautifulSoup(driver.page_source, 'lxml')
# locations = soup.find_all('div', jscontroller = 'AtSb') 
# clicks = driver.find_elements(By.CLASS_NAME,'cXedhc')

# count = 0 
# while count < len(locations) :
#     location = locations[count].text
#     location = location[:location.index('.')-1]

#     if "Sponsored" in location:
#         print("Skipped")
#     else:
#         clicks[count].click()
#         time.sleep(2)
#         soup = BeautifulSoup(driver.page_source, 'lxml')
#         phoneNumber = "No number present"
#         try:
#               phoneNumber = soup.find('a', jscontroller = 'LWZElb').text
#         except:
#               phoneNumber = "No number present"
            
#         website = soup.find('a', class_ = 'n1obkb mI8Pwc').get("href")
#         if not website:
#             print(location +", "+ phoneNumber +", No website present")
#         else:
#             print(location +", "+ phoneNumber +", "+ website)
        
#     count += 1